##  Modelo de word embbeding skip-gram com TensorFlow/Keras utilizando um corpus de questões aplicadas no Enem

In [4]:
#data
import numpy as np
import pandas as pd

#nlp
import nltk
import tensorflow as tf
from nltk.corpus import gutenberg
from string import punctuation
# import tf.keras
from keras.preprocessing import text
import spacy
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
import re

#ml
# Importa as classes necessárias do TensorFlow para criar camadas e modelos de rede neural.
from tensorflow.keras.layers import Concatenate, Dense, Embedding, Reshape
from tensorflow.keras.models import Model

#plot
import matplotlib.pyplot as plt

#stat
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import euclidean_distances

#aux
from IPython.display import display, clear_output
import os
import pickle

### Carregar o corpus

- Corpus obtido de https://github.com/johanessevero/classificacao_itens_enem_projeto_final_puc/tree/main

In [6]:
file = os.path.join('df_itens_geral_validos.csv')
df_itens = pd.read_csv(file, sep = ';', encoding = 'UTF-8')

In [7]:
df_itens = df_itens.loc[:, 'texto_questao']

In [8]:
df_itens = pd.DataFrame(df_itens)
df_itens

,texto_questao
0,O gráfico representa a relação entre o tamanho...
1,"Antes, eram apenas as grandes cidades que se a..."
2,A maioria das pessoas daqui era do campo. Vila...
3,Os lixões são o pior tipo de disposição final ...
4,O esquema representa um processo de erosão em ...
...,...
1917,Na montagem de uma cozinha para um restaurante...
1918,"Nas angiospermas, além da fertilização da oosf..."
1919,O sino dos ventos é composto por várias barra...
1920,O rompimento da barragem de rejeitos de minera...


### Pré-processar o texto

In [9]:
nlp = spacy.load("pt_core_news_md")

# realizar o pré-processamento de um texto
def pre_processa(text):
    
    stop_words = set(stopwords.words("portuguese"))
    #transformar em minúscula
    text = text.lower()
    text = re.sub(r"/\b(?:(?:https?|ftp):\/\/|www\.)[-a-z0-9+&@#\/%?=~_|!:,.;]*[-a-z0-9+&@#\/%=~_|]/i", ' ', text)
    text = re.sub(r'^(?!http)\S+', '', text)
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)
    text = re.sub(r'http[s]?:/(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)
    text = re.sub(r'www(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)
    #tokenizar
    text_tokens = word_tokenize(text)
    #remover stop words
    text_tokens = [word.lower() for word in text_tokens if word.lower() not in stop_words]
    #lematizar
    text_tokens = [nlp(word)[0].lemma_ for word in text_tokens]
    #remover pontuações
    text_tokens = [word for word in text_tokens if word not in string.punctuation]
    text = ' '.join(text_tokens)
    text = re.sub(r".disponível acesso \d{1,2} [a-zA-Z]+ \d{4}", '', text)
    text = re.sub(r".disponivel acesso \d{1,2} [a-zA-Z]+ \d{4}", '', text)
    text = re.sub(r"dísponivel acesso \d{1,2} [a-zA-Z]+ \d{4}", '', text)
    text = re.sub(r"\.dísponivel acesso \d{1,2} [a-zA-Z]+ \d{4}", '', text)
    text = re.sub(r"\.dísponivel acesso \d{1,2}", '', text)
    
    return text

In [10]:
for i, _ in enumerate(range(len(df_itens))):
    print(i)
    df_itens.loc[i, 'texto_tratado'] = pre_processa(df_itens.loc[i, 'texto_questao'])
    print(df_itens.loc[i, 'texto_tratado'])
    clear_output(wait=True)

1921
escola iniciar processo educativo implantação coleta seletivo edestino material reciclável atingir objetivo instituição planejou:1 sensibilizar comunidade escol ar desenvolver atividade sala extraclassede maneira contínuo 2 capacitar pessoal responsá vel limpeza escola quanto novosprocedimento adotar coleta seletivo e3 distribuir coletor material reciclável específico sala pátio outrosambiente acondicionamento resíduos.para completar ação propor ambiente escolar falta inserir noplanejamento arealizar campanha educativo sensibilização bairro vizinho parafortalecer coleta seletiva.bfirmar parceria prefeitura cooperativa catador recolhimentodo material reciclável destinação apropriada.corganizar visita s lixão aterro local identificar aspecto importantessobrir disposição final lixo.ddivulgar rádio local jorna l impresso rede social escola estárealizar coleta seletiva.ecolocar recipien te coletor lixo reciclável escola entregavoluntário população


In [11]:
texts = list(df_itens['texto_tratado'].values)

### Obter a coleção de skip-grams a partir do corpus

Um **"skip-gram"** é um par de palavras (a palavra alvo e a palavra de contexto) e um rótulo que indica se a palavra de contexto aparece dentro de uma **janela fixa de tamanho** em relação à palavra alvo no texto de entrada. Os skip-grams são usados como dados de treinamento para um modelo de word embbeding. Um "skip-gram" é uma unidade de dados usada em processamento de linguagem natural (NLP) e aprendizado de máquina para treinar modelos de representações vetoriais de palavras, conhecidos como modelos de word embbedings.

Cada skip-gram consiste em dois elementos principais:

- **Palavra Alvo (Target Word)**: É uma palavra específica que é escolhida como ponto central do skip-gram. O objetivo é aprender uma representação vetorial para essa palavra.
- **Palavra de Contexto (Context Word)**: É outra palavra que aparece nas proximidades da palavra alvo no texto. A proximidade é definida por uma janela de tamanho fixo. A palavra de contexto ajuda a capturar o contexto ou o significado da palavra alvo.
- **Rótulo (label)**: cada skip-gram possui um rótulo associado que indica se a palavra de contexto está dentro da janela de contexto da palavra alvo. Esse rótulo pode ser binário, indicando "sim" ou "não", dependendo se a palavra de contexto está ou não dentro da janela.

Imagine que se está trabalhando com um texto simples, como "O gato preto está na árvore." Neste contexto, um skip-gram poderia ser criado da seguinte forma: Palavra alvo: "preto". Palavra de contexto: "gato". Rótulo: "1" (indicando que "gato" é de fato uma palavra de contexto de "preto" dentro de uma janela de contexto específica) Isso representa a ideia de que, dentro de um determinado contexto, a palavra "gato" é uma palavra de contexto próxima à palavra "preto".

Esses skip-grams são criados a partir de um corpus de texto maior. O processo envolve varrer o texto e identificar pares de palavras alvo e de contexto dentro da janela definida.

Depois de criar uma **coleção de skip-grams**, eles são usados como dados de treinamento para um modelo de **modelo de word embbeding**. O objetivo do treinamento é ajustar os pesos do modelo de forma que as representações vetoriais das palavras sejam aprendidas de maneira a preservar informações semânticas e contextuais. Essas representações são então usadas em tarefas de NLP, como classificação de documentos, tradução automática e recomendação de palavras-chave, entre outras.

In [24]:
# Esta linha imprime o número total de linhas no corpus original
print('Total de linas no corpus:', len(texts))

# Esta linha imprime uma amostra de uma linha do corpus original
print('\nExemplo de linha:', texts[5])

# Esta linha cria um objeto tokenizer usando a classe 'Tokenizer' do módulo 'text'.
tokenizer = text.Tokenizer()

# Esta linha ajusta o tokenizer para os textos fornecidos em 'texts'. 
# Isso processará o texto e criará um índice para cada palavra única.
tokenizer.fit_on_texts(texts)

# Esta linha cria um dicionário que mapeia palavras para IDs (índices) com base no ajuste feito pelo tokenizer.
word_to_id = tokenizer.word_index

# Esta linha cria um dicionário que mapeia IDs (índices) de volta para palavras, invertendo o mapeamento em 'word_to_id'.
id_to_word = {v: k for k, v in word_to_id.items()}

# Esta linha calcula o tamanho do vocabulário somando 1 ao número de palavras únicas. 
# O '+1' é para acomodar um possível token para palavras fora do vocabulário.
vocab_size = len(word_to_id) + 1

# Esta linha define o tamanho da representação vetorial (embedding) para cada palavra como 100. 
# Isso significa que cada palavra será representada por um vetor de 100 dimensões.
embedding_size = 300

# Esta linha cria uma lista 'word_ids' que contém uma lista de IDs de palavras para cada documento (sentença) em 'texts'.
# Ele utiliza a estrutura de compreensão de lista para iterar sobre cada documento, que é primeiro convertido em uma lista de palavras com 'text.text_to_word_sequence(doc)',
# e então para cada palavra na lista de palavras, ele encontra o ID correspondente usando o dicionário 'word_to_id'.
word_ids = [[word_to_id[w] for w in text.text_to_word_sequence(doc)] for doc in texts]

# Esta linha imprime o tamanho do vocabulário, ou seja, o número total de palavras únicas no corpus processado da Bíblia.
print('Tamanho do vocabulário:', vocab_size)

# Esta linha imprime uma amostra das 10 primeiras entradas do dicionário que mapeia palavras para IDs, fornecendo uma visão do mapeamento palavra-ID.
print('Exemplo de vocabulário:', list(word_to_id.items())[:10])

Total de linas no corpus: 1922

Exemplo de linha: processo erosivo concentrar encosta principalmente motivar água vento entanto reflexo sentido área baixar onde geralmente ocupação urbano exemplo de esse reflexo vida cotidiano muito cidade brasileiro grande ocorrência enchente rio assorear comportar menos água leito b contaminação população sedimento trazir Rio carregar matéria orgânica c desgaste solo área urbano causar redução escoamento superficial pluvial encosta d grande facilidade captação água potável abastecimento público grande efeito escoamento sobre infiltração aumento incidência doença amebíase população urbano decorrência escoamento água poluir topo encosta
Tamanho do vocabulário: 26072
Exemplo de vocabulário: [('b', 1), ('c', 2), ('d', 3), ('e', 4), ('se', 5), ('esse', 6), ('1', 7), ('de', 8), ('texto', 9), ('adaptar', 10)]


In [35]:
# Esta linha cria pares de skip-grams para o corpus processado da Bíblia. 
# Para cada sequência de palavras representada por 'wid' em 'word_ids',
# ela gera pares de skip-grams usando a função 'tf.keras.preprocessing.sequence.skipgrams'.
skip_grams = [tf.keras.preprocessing.sequence.skipgrams(wid, vocabulary_size=vocab_size, window_size=10) for wid in word_ids]

# Esta linha extrai os pares e rótulos do primeiro conjunto de skip-grams gerado anteriormente.
pairs, labels = skip_grams[0][0], skip_grams[0][1]

# Este loop itera sobre os 10 primeiros pares de skip-grams e seus rótulos, e os imprime formatados.
for i in range(5):
    print("({:s} ({:d}), {:s} ({:d})) -> {:d}".format(
          id_to_word[pairs[i][0]], pairs[i][0], 
          id_to_word[pairs[i][1]], pairs[i][1], 
          labels[i]))

(gráfico (231), gol (1315)) -> 0
(existência (872), territorial (1081)) -> 1
(apresentar (19), c (2)) -> 1
(concentraçao (11204), evidenciar (537)) -> 1
(gráfico (231), radicalizar (22290)) -> 0


### Construir a arquitetura do modelo de word embedding skip-gram

As **representações vetoriais/embeddings de palavras** são representações numéricas de palavras em um espaço vetorial contínuo de dimensões. Elas são capazes de representar palavras de forma que as relações semânticas e sintáticas entre elas sejam capturadas de maneira significativa. Essas representações vetoriais permitem que modelos de aprendizado de máquina compreendam e manipulem palavras de forma mais eficaz em tarefas de Processamento de Linguagem Natural (NLP) e podem ser usadas em aplicações como tradução automática, análise de sentimento, recomendação de texto e muito mais.

Um **embedding de palavras**, é uma forma de representar palavras em um espaço matemático de várias dimensões, onde cada dimensão corresponde a algum aspecto do significado da palavra. Essas representações têm a capacidade única de capturar relações semânticas entre palavras. 

Por exemplo, suponhamos que estejamos trabalhando com representações vetoriais de palavras em um espaço muldimensional hipotético.
Considere as palavras "rainha" e "rei". Se representarmos essas palavras como vetores nesse espaço, podemos observar que elas têm uma relação semântica forte na dimensão "realeza". Isso significa que essas palavras têm valores significativos nessa dimensão, indicando que estão relacionadas à realeza.
Além disso, na mesma representação vetorial, essas palavras também têm valores significativos na dimensão "pessoa", pois ambas se referem a indivíduos.
Portanto, as representações vetoriais conseguem capturar a relação semântica entre "rainha" e "rei" ao mostrar que ambas têm similaridade na dimensão "realeza" e "pessoa".

Agora, considere as palavras "cachorro" e "gato". Essas palavras também podem ter relações semânticas em representações vetoriais, mas de uma maneira diferente.
Se essas palavras foram treinadas em um contexto onde se referem a animais de estimação, é provável que compartilhem uma relação semântica forte na dimensão "animal de estimação". Isso significa que seus vetores teriam valores significativos nessa dimensão, indicando que ambas as palavras estão relacionadas como animais de estimação.
Nesse contexto, o espaço vetorial é capaz de capturar a relação semântica e contextual entre "cachorro" e "gato" na dimensão específica de "animal de estimação".

Um **modelo de Word Embedding Skip-gram** é um algoritmo de aprendizado de máquina utilizando redes neurais utilizado para aprender os word embeddings a partir de grandes conjuntos de texto. Esse modelo pode utilizar coleções de skip-grams como dados de treinamento. A **arquitetura do modelo skip-gram** consiste em uma camada de entrada, uma camada de saída e uma camada oculta. No modelo que iremos construir, a camada de entrada é um skip-grams obtido a partir de cada sentença do corpus e a camada de saída é um valor de probabilidade de a palavra de contexto está na janela de contexto da palavra alvo. A camada oculta representa o word embedding da entrada aprendida durante o treinamento. O modelo skip-gram usa uma **rede neural feedforward** com uma única camada oculta.

Uma **rede neural feedforward**, também conhecida como rede neural de alimentação direta ou perceptron multicamadas, é um tipo fundamental de arquitetura de rede neural artificial. Nesse tipo de rede os dados fluem de uma camada de entrada através de uma ou mais camadas ocultas até a camada de saída, sem retroalimentação. A rede neural feedforward é amplamente utilizada em tarefas de aprendizado supervisionado, como classificação e regressão, e é conhecida por sua capacidade de aprender representações complexas de dados.

A arquitetura da rede neural construída será:

- **Camada de entrada para a palavra alvo**: É uma camada de entrada para a palavra alvo;
- **Camada de entrada para a palavra de contexto**: É uma camada de entrada para a palavra de contexto;
- **Camada de embedding para as palavras alvo**: Esta camada mapeia as palavras alvo para vetores densos de dimensão $n$.
- **Camada de embedding para as palavras de contexto**: Esta camada mapeia as palavras de contexto para vetores densos de dimensão $n$.
- **Camada densa de saída:**: Esta é uma camadav que produz a saída final do modelo com ativação sigmoide;

A arquitetura será montada de modo que os lotes passados a cada iteração são a coleção de skip-grams obtidos de cada sentença do corpus. Ao final do treinamento não estamos interessados nas saídas da rede neural, mas nos **pesos aprendidos da camada de embedding para as palavras alvos**. Esses pesos são os word embeddings das palavras e as representações contextuais aprendidas entre as palavras. A rede é treinada uma tarefa "falsa", pois o objetivo é apenas aprender os pesos da camada oculta que são na verdade os "vetores de palavras". Nessa arquitetura as entradas são skip-grams. Como as sentenças são de tamanho diferente então o tamanho do vetor de palavras alvo e de contexto varia de acordo com as sentenças.

<img src = "skip-gram_model.png">

O tamanho do vocabulário é o número de palavras únicas no conjunto de dados. Neste caso, tem-se um vocabulário com 26072 palavras únicas. Cada palavra é mapeada para um vetor densamente representado em um espaço de embedding com 300 dimensões. Para cada dimensão do vetor de embedding, a camada de embedding aprende um conjunto de pesos (parâmetros) para cada palavra no vocabulário. Calculando o número total de parâmetros treináveis na camada de embedding: 26072 x 300 = 15643801. A camada de embedding possui 15643801 parâmetros treináveis porque mapeia cada uma das 26072 palavras únicas no vocabulário para um vetor de 300 dimensões, e cada dimensão desse vetor tem seu conjunto de pesos (parâmetros) para serem aprendidos durante o treinamento da rede neural.

In [15]:
target_word_input = tf.keras.Input(shape=(1,))
context_word_input = tf.keras.Input(shape=(1,))

Esta linha de código está criando um **ponto de entrada (input)** para o TensorFlow/Keras e está definindo as características desse ponto de entrada:

- **target_word_input**: Esta parte da linha está atribuindo um nome ao ponto de entrada do modelo. O nome, neste caso, é target_word_input. Esse nome é usado posteriormente para referenciar esse ponto de entrada ao construir o modelo.

- **tf.keras.Input(...)**: Esta é a função que cria o ponto de entrada. Ela é uma parte fundamental do Keras e é usada para definir as entradas do modelo.

- **shape=(1,)**: Esta parte define a forma (shape) das entradas que serão alimentadas neste ponto de entrada. No contexto deste código, shape=(1,) significa que as entradas serão vetores unidimensionais com um único elemento em cada vetor.

A linha de código está criando um ponto de entrada chamado target_word_input que espera receber vetores unidimensionais com um único elemento cada. Isso é útil quando estamos construindo modelos que envolvem representações vetoriais de palavras, onde cada palavra é representada como um vetor com uma dimensão específica (neste caso, 1). Esse ponto de entrada será usado posteriormente para fornecer palavras-alvo como entrada para o modelo.

In [16]:
target_word_model = Embedding(vocab_size, embedding_size, embeddings_initializer="glorot_uniform")(target_word_input)
context_word_model = Embedding(vocab_size, embedding_size, embeddings_initializer="glorot_uniform")(context_word_input)

Esta linha de código está relacionada à criação de uma **embedding layer** em uma rede neural usando TensorFlow/Keras.

- **target_word_model**: Isso cria uma variável chamada target_word_model que será usada para representar a camada de embedding da palavra alvo (target word embedding). Essa variável será usada posteriormente para conectar outras camadas e construir a arquitetura do modelo.

- **Embedding(...)**: Cria-se uma camada de embedding usando a função Embedding do TensorFlow/Keras. A camada de embedding é usada para mapear palavras (ou tokens) para vetores densos de números reais. Vamos analisar os argumentos passados para a função Embedding:

     - **vocab_size**: Este é o primeiro argumento e representa o tamanho do vocabulário. Em outras palavras, é o número total de palavras únicas que a rede neural espera encontrar durante o treinamento. Cada palavra única receberá uma representação única em forma de vetor.

    - **embedding_size**: Este é o segundo argumento e define a dimensão do espaço de embedding. Cada palavra será representada por um vetor de embedding_size dimensões. Isso determina o tamanho da representação vetorial para cada palavra. Um valor comum para embedding_size é 100 ou 300.

    - **embeddings_initializer**: Este é um parâmetro opcional que define como os vetores de embedding são inicializados. No seu exemplo, "glorot_uniform" é usado, que é um método de inicialização comum que ajuda na convergência mais eficaz do treinamento.

    - **(target_word_input)**: A camada de embedding é então conectada à camada de entrada target_word_input. Isso estabelece que a camada de embedding receberá as palavras-alvo como entrada, e as transformará em vetores densos com base nas configurações fornecidas nos argumentos anteriores.

In [17]:
target_word_model = Reshape((embedding_size,))(target_word_model)
context_word_model = Reshape((embedding_size,))(context_word_model)

Essa linha de código faz parte do **processo de manipulação dos dados de saída da camada de embbedding** da palavra de contexto em um formato adequado para uso subsequente.

- **context_word_model**: Esta é a variável que representa a saída da camada de embedding da palavra de contexto, criada anteriormente. Ela contém as representações vetoriais das palavras de contexto.

- **Reshape((embedding_size,))**: Aplica-se a operação de remodelação (reshape) à saída da camada de embedding. A função Reshape é usada para alterar a forma (shape) dos dados. No seu caso, estamos especificando que queremos remodelar a saída para ter uma nova forma com (embedding_size,). Isso significa que queremos que os dados passem de uma matriz para um vetor unidimensional, onde cada elemento do vetor corresponderá a uma dimensão do espaço de embedding.

    - **embedding_size**: Este é o tamanho da nova forma que estamos definindo. É igual à dimensão do espaço de embedding e determina o tamanho do vetor resultante.
    - **(context_word_model)**: aplica-se a operação de remodelação à variável context_word_model, que contém a saída da camada de embedding da palavra de contexto.

Essa linha de código realiza o redimensionamento da saída da camada de embedding da palavra de contexto de uma matriz para um vetor unidimensional, garantindo que os dados tenham a forma correta para serem usados nas operações subsequentes da rede neural. Isso é frequentemente necessário para que os dados possam ser alimentados em outras camadas ou serem usados em operações matriciais adequadas ao restante da arquitetura da rede neural.

In [18]:
merged = Concatenate(axis=1)([target_word_model, context_word_model])

Essa linha de código está relacionada à criação de uma **representação combinada das palavras alvo e de contexto** para alimentar posteriormente em camadas adicionais da rede neural.

- **merged**: Esta é a variável que receberá a representação combinada das palavras alvo e de contexto. Essa representação combinada é gerada pela concatenação das representações vetoriais das palavras alvo e de contexto.

- **Concatenate(axis=1)**: Aqui, estamos criando uma camada de concatenação (Concatenate) usando a função Concatenate do TensorFlow/Keras. Esta camada é responsável por combinar (concatenar) os vetores das palavras alvo e de contexto em uma única representação combinada. O argumento axis=1 especifica que a concatenação deve ser realizada ao longo do eixo 1, ou seja, as representações das palavras são empilhadas horizontalmente.

- **([target_word_model, context_word_model])**: Dentro dos parênteses, estamos passando uma lista contendo as duas representações vetoriais que desejamos concatenar: target_word_model e context_word_model. Essas são as saídas das camadas de embedding correspondentes às palavras alvo e de contexto.

A linha de código, portanto, concatena as representações vetoriais das palavras alvo e de contexto ao longo do eixo 1, criando uma representação combinada que incorpora informações das duas palavras. Isso é útil em arquiteturas de redes neurais que buscam capturar relações semânticas entre palavras alvo e de contexto, como em modelos de linguagem e sistemas de recomendação baseados em palavras-chave. A saída dessa concatenação, armazenada na variável merged, pode ser alimentada em camadas subsequentes da rede neural para realizar tarefas específicas, como classificação ou regressão.

In [19]:
output = Dense(1, kernel_initializer="glorot_uniform", activation="sigmoid")(merged)

Esta linha de código está relacionada à criação da **camada de saída de uma rede neural**.

- **output**: Esta é a variável que receberá a saída final da rede neural. Em problemas de aprendizado supervisionado, como classificação ou regressão, esta variável geralmente contém as previsões feitas pelo modelo para as entradas dadas.

- **Dense(1, ...)**: Aqui, estamos criando uma camada densa (fully connected) usando a função Dense do TensorFlow/Keras. A camada densa é uma camada neural que conecta cada neurônio (unidade) à saída de todos os neurônios da camada anterior.

- **1:** O primeiro argumento especifica o número de unidades (neurônios) na camada densa de saída. Neste caso, estamos usando 1, o que indica que a camada de saída terá uma única unidade de saída. Isso é comum em tarefas de classificação binária, onde estamos tentando prever uma única saída (por exemplo, uma probabilidade).
kernel_initializer="glorot_uniform": O argumento kernel_initializer especifica o método de inicialização dos pesos da camada densa. "glorot_uniform" é um método comum de inicialização que define os pesos iniciais de forma apropriada para ajudar na convergência eficaz do treinamento.

- **activation="sigmoid"**: O argumento activation especifica a função de ativação a ser aplicada à saída da camada densa. Neste caso, estamos usando a função de ativação sigmoidal ("sigmoid"). A função sigmoidal é comumente usada em tarefas de classificação binária, pois comprime a saída para um valor entre 0 e 1, representando uma probabilidade.

- **(merged)**: Finalmente, estamos conectando a camada densa à saída da camada anterior, que é a representação combinada das palavras alvo e de contexto, armazenada na variável merged. Isso estabelece que a camada densa receberá essa representação combinada como entrada e produzirá a saída final da rede neural.

Portanto, a linha de código cria a camada de saída da rede neural, que produz uma única saída com base na representação combinada das palavras alvo e de contexto. Essa saída é tipicamente usada para fazer previsões ou estimativas em uma tarefa de aprendizado de máquina, dependendo do problema em questão.


In [20]:
model = Model(inputs=[target_word_input, context_word_input], outputs=output)

Esta linha de código está relacionada à criação e definição do **modelo de rede neural**.

- **model**: Esta é a variável que armazenará o modelo de rede neural que estamos construindo. Um modelo é uma representação abstrata de uma rede neural que consiste em várias camadas conectadas.

- **Model(inputs=[target_word_input, context_word_input], outputs=output)**: Cria-se um objeto Model usando a classe Model do TensorFlow/Keras. Este objeto representa o modelo de rede neural que se está construindo. 
    - **inputs=[target_word_input, context_word_input]**: Isso especifica as camadas de entrada do modelo. Em particular, estamos fornecendo uma lista de camadas de entrada. Neste caso, temos duas camadas de entrada: target_word_input e context_word_input. Cada uma dessas camadas representa um ponto de entrada para diferentes tipos de dados de entrada, como palavras alvo e palavras de contexto.
    - **outputs=output**: Isso especifica a camada de saída do modelo. Estamos usando a variável output, que representa a camada de saída que foi definida anteriormente. Essa camada de saída produzirá as previsões da rede neural com base nos dados de entrada.

Essa linha de código cria um modelo de rede neural que recebe duas camadas de entrada diferentes (target_word_input e context_word_input) e produz saídas com base na camada de saída (output). Isso define a arquitetura da rede neural, estabelecendo como os dados fluem do ponto de entrada para a camada de saída através de várias camadas intermediárias, se houver. Depois de criar o modelo, ele pode ser treinado com dados de treinamento e usado para fazer previsões em dados de teste.

In [25]:
model.compile(loss="binary_crossentropy", optimizer="adam")

Esta linha de código está relacionada à configuração do **processo de treinamento de um modelo de rede neural** em TensorFlow/Keras.

- **model.compile(...)**: Este método é usado para compilar o modelo de rede neural que foi definido anteriormente. Durante a compilação, você especifica várias configurações que afetam o processo de treinamento do modelo.

- **loss="binary_crossentropy"**: é uma **função de perda** utilizada em problemas de classificação binária. Ela mede a diferença entre a previsão do modelo e o valor alvo (verdadeiro). Isso é feito calculando a entropia cruzada (cross-entropy) entre a distribuição de probabilidade predita pelo modelo e a distribuição de probabilidade real..

- **optimizer="adam"**: O segundo argumento, optimizer, especifica o **otimizador** que será usado para ajustar os pesos da rede neural durante o treinamento. No seu exemplo, está configurado como "adam", que é um otimizador popular e eficaz. O otimizador Adam é uma variante do gradiente descendente estocástico (SGD) que adapta dinamicamente as taxas de aprendizado para cada parâmetro, tornando o treinamento mais eficiente.

Após a compilação, o modelo estará pronto para o treinamento. Durante o treinamento, ele usará a função de perda "Cross Entropy Loss" para calcular o erro entre as previsões e os rótulos verdadeiros, e o otimizador "adam" ajustará os pesos da rede para minimizar essa função de perda. O processo de treinamento envolverá iterar sobre os dados de treinamento, calcular gradientes, atualizar os pesos e repetir esse processo por várias épocas até que o modelo atinja um desempenho desejado.

A Log Loss, também conhecida como Cross Entropy Loss, é uma função de perda utilizada em problemas de classificação, especialmente em modelos de aprendizado de máquina para classificação binária ou multiclasse. Ela mede a performance de um modelo de classificação, comparando as previsões do modelo com as verdadeiras classes dos exemplos de treinamento.

A fórmula geral para a Log Loss em um problema de classificação binária é:

- Log Loss $= −\frac{1}{N}\sum_{N}^{i=1}(y⋅log⁡(p_i)+(1−y_i)⋅log⁡(1−p_i))$
    - $N$: é o número total de exemplos de treinamento,
    - $y_i$: é a verdadeira classe do exemplo $i$ (1 se pertence à classe positiva, 0 se pertence à classe negativa),
    - $p_i$: é a probabilidade predita pelo modelo de que o exemplo $i$ pertence à classe positiva.

Em um problema de classificação multiclasse, a fórmula é generalizada para várias classes.

Exemplo:

Suponha que temos um problema de classificação binária onde estamos tentando prever se um e-mail é spam ou não com base em várias características. Vamos calcular a Log Loss para um único exemplo:

- Verdadeira classe $(y): 1$ spam;
- Probabilidade predita $(p): 0.8$

Log Loss = $−(1⋅log⁡(0.8)+(1−1)⋅log⁡(1−0.8)) =  0.22$

O resultado seria um valor positivo, indicando quão bem as probabilidades preditas correspondem às classes reais. O objetivo é minimizar a Log Loss durante o treinamento do modelo.
<p>O objetivo é minimizar a Log Loss. Quanto menor o valor, melhor o desempenho do modelo. Um Log Loss de 0 indica que as previsões são perfeitas, enquanto valores mais altos indicam um desempenho pior.
<p>A Log Loss é sempre um valor positivo. O mínimo é 0, e não há limite superior. Em prática, valores próximos de 0 indicam previsões muito precisas, enquanto valores mais altos indicam maior incerteza nas previsões.
<p>A interpretação intuitiva é que a Log Loss penaliza mais fortemente as previsões incorretas que são feitas com alta confiança. Se o modelo faz uma previsão errada com uma probabilidade próxima de 1, a penalidade será significativamente maior do que se a probabilidade predita for próxima de 0.
<p>A Log Loss é útil para comparar diferentes modelos. Ao avaliar vários modelos, o modelo com a Log Loss mais baixa é geralmente considerado o melhor, desde que seja uma comparação justa em termos de conjunto de dados e características.
Em um problema de classificação binária, a Log Loss para uma única instância pode ser interpretada como a quantidade média de surpresa (ou incerteza) associada à previsão do modelo. Se o modelo está muito confiante e errado, a Log Loss será alta.

In [26]:
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 1)]                  0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 1)]                  0         []                            
                                                                                                  
 embedding (Embedding)       (None, 1, 300)               7821600   ['input_1[0][0]']             
                                                                                                  
 embedding_1 (Embedding)     (None, 1, 300)               7821600   ['input_2[0][0]']             
                                                                                              

Esta linha imprime um resumo do modelo, mostrando a arquitetura da rede neural, suas camadas e o número de parâmetros.

Representação de um modelo de rede neural em TensorFlow/Keras, exibida no formato de um resumo de modelo. 

- **Model: "model"**: Isso indica o nome do modelo, que é simplesmente "model" neste caso.
- **Layer (type)**: Aqui, cada linha representa uma camada no modelo, com informações sobre o tipo da camada e outras informações relevantes.
- **Output Shape:** Essa coluna mostra a forma da saída produzida por cada camada. Por exemplo, "(None, 1)" significa que a camada produz uma saída com forma (None, 1), onde "None" geralmente indica que a dimensão pode variar dependendo do tamanho do lote de entrada.
- **Param #:** Esta coluna exibe o número de parâmetros treináveis (pesos) em cada camada. Esses são os valores que o modelo aprende durante o treinamento para fazer previsões.
- **Connected to**: Esta coluna mostra quais camadas estão conectadas à camada atual. Por exemplo, "['input_1[0][0]']" indica que a camada atual está conectada à camada de entrada "input_1". A notação entre colchetes "[0][0]" é usada para identificar a conexão específica dentro da camada de entrada.

- **Summary**: Isso mostra um resumo geral do modelo, incluindo o número total de parâmetros, parâmetros treináveis e parâmetros não treináveis. Aqui estão as camadas específicas no modelo e uma explicação detalhada:

    - **input_1 (InputLayer)**: Esta é a camada de entrada chamada "input_1". Ela espera dados de entrada com forma (None, 1). É uma camada de entrada para a palavra alvo.
    - **input_2 (InputLayer)**: Esta é a segunda camada de entrada chamada "input_2", também esperando dados de entrada com forma (None, 1). É uma camada de entrada para a palavra de contexto.
    - **embedding (Embedding)**: Esta é uma camada de embedding (embedding) chamada "embedding". Ela mapeia as palavras alvo para vetores densos de dimensão 300. O número de parâmetros treináveis nesta camada é 7821600.
    - **embedding_1 (Embedding)**: Esta é outra camada de embedding chamada "embedding_1" para as palavras de contexto, com a mesma dimensão de saída de 300. O número de parâmetros treináveis nesta camada também é 7821600.
    - **reshape (Reshape)**: Esta é uma camada de remodelação chamada "reshape" que altera a forma da saída da camada de embedding para (None, 300). Não possui parâmetros treináveis.
    - **reshape_1 (Reshape)**: Similar à camada anterior, esta é uma camada de remodelação chamada "reshape_1" para as palavras de contexto, com saída também em (None, 300).
    - **concatenate (Concatenate)**: Esta é uma camada de concatenação chamada "concatenate" que combina as representações das palavras alvo e de contexto. A saída é (None, 200) devido à concatenação das representações de 300 dimensões de cada uma das camadas reshape anteriores.
    - **dense (Dense):** Esta é uma camada densa chamada "dense" que produz a saída final do modelo, que é uma única unidade (None, 1) com ativação sigmoide. Ela possui 201 parâmetros treináveis.
    - **Total params:** Esta seção resume o número total de parâmetros no modelo, que é 3087602. Isso inclui todos os parâmetros treináveis em todas as camadas.
    - **Trainable params**: Indica o número de parâmetros que podem ser treinados durante o treinamento do modelo. No total, todos os parâmetros são treináveis.
    - **Non-trainable params**: Indica o número de parâmetros que não são treináveis. No caso deste modelo, todos os parâmetros são treináveis, então esse valor é zero.

### Treinando o modelo

In [27]:
losses = []
# treina o modelo em skip-grams
# Este é um loop que itera pelas épocas do treinamento. O treinamento é executado por 5 épocas, numeradas de 1 a 5.
for epoch in range(0, 20):
    # Inicializa a variável total_loss como zero para acompanhar a perda total durante a época atual.
    total_loss = 0
    # Inicia um loop interno para iterar pelos pares de skip-grams. Enumerate é usado para obter tanto o índice i quanto o elemento elem de skip_grams.
    losses_epoch = []
    for i, elem in enumerate(skip_grams):
        # Extrai as primeiras palavras (target words) dos pares skip-gram e as converte em um array numpy de inteiros com o tipo de dado 'int32'.
        skip_first_elem = np.array(list(zip(*elem[0]))[0], dtype='int32')
        # Extrai as segundas palavras (context words) dos pares skip-gram e as converte em um array numpy de inteiros com o tipo de dado 'int32'.
        skip_second_elem = np.array(list(zip(*elem[0]))[1], dtype='int32')
        # Extrai os rótulos (labels) dos pares skip-gram e os converte em um array numpy de inteiros com o tipo de dado 'int32'.
        labels = np.array(elem[1], dtype='int32')
        # Cria uma lista X que contém as duas partes do skip-gram: as primeiras palavras e as segundas palavras.
        X = [skip_first_elem, skip_second_elem]
        #print('Tamanho do input 1 (palavras alvo) e input 2 (palavras de contexto): ', len(X[0]))
        # Cria uma variável Y para armazenar os rótulos dos pares skip-gram.
        Y = labels
        #print('Tamanho da saída', len(Y))
        # Exibe uma mensagem a cada 10.000 iterações para indicar o progresso do processamento dos pares skip-gram.
        #if i % 10000 == 0:
            #print('Processed {} skip-gram pairs'.format(i))
        # Realiza o treinamento da rede neural em lote (batch) com os dados X e Y e acumula a perda resultante na variável total_loss.
        loss = model.train_on_batch(X,Y)  
        losses_epoch.append(loss)
        #clear_output(wait=True)
    losses.append(losses_epoch)
    # Após o término de um loop interno (processamento de todos os skip-grams na época atual), exibe o número da época e a perda total durante essa época. Isso ajuda a monitorar o progresso do treinamento da rede neural.
    print('Época: {} Perda: {}'.format(epoch, np.mean(losses[epoch])))

Época: 0 Perda: 0.4958029375426107
Época: 1 Perda: 0.41848964948870016
Época: 2 Perda: 0.391965904644265
Época: 3 Perda: 0.3748078024505204
Época: 4 Perda: 0.36325369370591504
Época: 5 Perda: 0.35516248875211354
Época: 6 Perda: 0.3500228994114094
Época: 7 Perda: 0.34636218884608994
Época: 8 Perda: 0.34354393954052515
Época: 9 Perda: 0.3412125087367384
Época: 10 Perda: 0.3392675905784012
Época: 11 Perda: 0.33758142324596735
Época: 12 Perda: 0.3361529082633578
Época: 13 Perda: 0.334854547752302
Época: 14 Perda: 0.3337359955450995
Época: 15 Perda: 0.33280972575887063
Época: 16 Perda: 0.33204105508811266
Época: 17 Perda: 0.3314024476212344
Época: 18 Perda: 0.3309915700266438
Época: 19 Perda: 0.3306710871709869


<code>skip_first_elem = np.array(list(zip(*elem[0]))[0], dtype='int32')</code>

Essa linha está relacionada à extração das primeiras palavras (target words) dos pares skip-gram e à conversão dessas palavras em um array numpy com tipo de dados 'int32':

- **elem[0]**: É um elemento da lista skip_grams, que é um par de palavras (skip-gram) representado como uma lista. - **elem[0]** acessa a primeira parte do skip-gram, que é uma lista contendo as palavras alvo e de contexto.
- **zip( \*elem[0])**: A função zip é usada para criar uma lista de tuplas, onde cada tupla contém as palavras alvo e de contexto correspondentes em cada par de skip-gram. O  \* antes de elem[0] é usado para descompactar a lista, para que o zip funcione corretamente.
- **list(zip( \*elem[0]))**: Converte o resultado de zip em uma lista.
- **[0]**: Indexa a lista resultante para acessar a primeira palavra em cada tupla, que é a palavra alvo.
- **np.array(...)**: Converte a lista das primeiras palavras em um array numpy.
- **dtype='int32'**: Define o tipo de dados do array numpy como 'int32', ou seja, um tipo de dado de número inteiro de 32 bits.

No final, a variável skip_first_elem conterá um array numpy que armazena as primeiras palavras (target words) dos pares skip-gram, e essas palavras são representadas como números inteiros de 32 bits. Esse processo é útil para preparar os dados de entrada da rede neural durante o treinamento.

In [1]:
# obtém os embeddings para as palavras do vocabulário
weights = model.layers[2].get_weights()[0]
weights.shape

NameError: name 'model' is not defined

In [31]:
# Nesta linha, a função euclidean_distances é usada para calcular a matriz de distância euclidiana. Ela recebe uma matriz de entrada weights. Cada linha dessa matriz representa o vetor de peso (embedding) de uma palavra. A função calcula a distância euclidiana entre todos os pares de vetores de peso no conjunto de dados e armazena essas distâncias na matriz distance_matrix.
distance_matrix = euclidean_distances(weights)
print(distance_matrix.shape)

#.argsort(): O método argsort() é usado para obter os índices que classificariam os valores em ordem crescente. No nosso caso, ele retorna os índices que classificariam as distâncias euclidianas da palavra 'água' em ordem crescente, ou seja, os índices das palavras mais próximas às mais distantes.
similar_words = {search_term: [id_to_word[idx] for idx in distance_matrix[word_to_id[search_term]-1].argsort()[1:40]+1] 
                   for search_term in ['manga']}

# imprime as 40 palavras mais similares
print(similar_words)

(26072, 26072)
{'manga': ['aﬁar', 'jabuticaba', 'nuanceseuforia', 'formigar', 'triclosan', 'incluso', 'sintáctica', '2umidade', 'laterizar', 'frever', 'ingressar', 'preferenciais', 'lã', 'glicogênio', '28disponível', 'barulhenta', 'silenciosa', 'questione', 'piruvatoudp', 'frevurar', 'caçamba', 'rehabs', 'img0', 'espelhar', 'tropóﬁla', 'benéﬁco', 'agpaseglicogêniodgattriacilglicerolco2o2h2ophaapoli', 'uencer', 'páginas', 'legislativa', 'ﬁnalíssimo', 'beat', '000reservatório', 'paratireoide', 'reinvenção', 'infogns', 'intruso', 'bizarro', 'ooci']}


In [32]:
distance_matrix[word_to_id['matemática']-1][3648]

3.2560778

In [33]:
distance_matrix[word_to_id['matemática']-1].argsort()[1:20]+1

array([ 383,  110,  473,   71,  200,   49,  125,  293,   47, 1710,  358,
         68,  281, 8056,  233,  150,  181,  266,  139], dtype=int64)

### Referências

- https://towardsdatascience.com/nlp-101-word2vec-skip-gram-and-cbow-93512ee24314
- https://www.scaler.com/topics/nlp/skip-gram-model/
- https://github.com/johanessevero/classificacao_itens_enem_projeto_final_puc/tree/main
- https://www.datarobot.com/blog/introduction-to-loss-functions/#:~:text=At%20its%20core%2C%20a%20loss,ll%20output%20a%20lower%20number.